<a href="https://colab.research.google.com/github/NVHien10/BD_DQ/blob/main/BD_DotQuy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
print("hello")

hello


In [ ]:
!apt-get install git -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [1]:
!git clone https://github.com/NVHien10/BD_DQ.git

Cloning into 'BD_DQ'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (39/39), 19.50 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
%cd BD_DQ/

/content/BD_DQ


In [4]:
!ls

BD_DotQuy.ipynb  README.md


In [8]:
!git checkout main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [16]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [17]:
%cd BD_DQ/

/content/BD_DQ


In [14]:
!git add .


In [17]:
!git commit -m " push code"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [16]:
!git config --global user.email "anhbahien123321@gmail.com"
!git config --global user.name "VinhHien03"

In [19]:
!git branch

  HienA
* main
